# Solve RCPSP by linear programming: MILP

We admit that you followed the [first notebook](RCPSP%20%231%20Introduction.ipynb) that introduced you all the basics for RCPSP Problems, on which we will apply here linear programming.

Linear programming is a paradigm to model optimisation problem where the objective function and constraints are linear in terms of the variables : 

$ y = max_{x}(c^t.x) $
such that $A.x \leq b $

$x$ can be either floating or integer values, which explains the Mixed Integer Linear Programming denomination.

In [ ]:
import logging

import nest_asyncio

# patch asyncio so that applications using async functions can run in jupyter
nest_asyncio.apply()

# set logging level
logging.basicConfig(level=logging.INFO)

## Parsing model

In [ ]:
from discrete_optimization.rcpsp.rcpsp_parser import get_data_available, parse_file

# Parse some rcpsp file
filepath = [f for f in get_data_available() if "j601_1.sm" in f][0]
rcpsp_problem = parse_file(filepath)

## LP Solver 

In [ ]:
from discrete_optimization.rcpsp.solver.rcpsp_lp_solver import LP_RCPSP, ParametersMilp

We have a few MILP model/solver available for single mode RCPSP, ```LP_RCPSP``` and ```LP_MRCPSP``` are using [mip](https://www.python-mip.com) python library and are adapted to single and multimode RCPSP.
```LP_MRCPSP_GUROBI``` is using gurobi solver and can be used if you have the required licence. In this notebook we'll use open source solver ```CBC``` via the `mip` based solver.

The LP modelling of the problem is based on time-indexed binary variable : $x[task, time]$ indicates if the given task is started at the given time.
Constraints of *RCPSP* can be then written with a bit of effort. The LP models are written in ```model.init_model``` function of the solvers.

If we note $c(task, r)$ the consumption of ressource $r$ of a task and $cap(r)$ the capacity of the resource $r$ : 

- Precedence constraints 
$\forall t_1, \forall t_2 \in \text{successors}(t_1) \sum_{t} t.x[t_2,t] \geq \sum_{t} t.x[t_1,t]$
- Cumulative constraints
$\forall r \in \text{ressources}, \forall t\in[0,horizon] \sum_{task}c(task,r)*\sum_{t'\in[t-dur(task)+1, t]}x[task,t'] \leq cap(r)$
Here the second sum represents the fact that the task is running at time $t$ and thus is contributing by $c(task,r)$ to the resource co


In [ ]:
solver = LP_RCPSP(problem=rcpsp_problem)
parameters_milp = ParametersMilp.default()
solver.init_model(greedy_start=True)
results = solver.solve(parameters_milp=parameters_milp)

### Plot the quality of solutions found by LP solver

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1)
ax.plot([x[1] for x in results.list_solution_fits[::-1]], marker="o")
ax.set_ylabel("- makespan")
ax.set_xlabel("# solution found")
plt.show()

In [ ]:
# Retrieve the best_solution, fit found by the solver.
best_solution, fit = results.get_best_solution_fit()
# Print fitness
print(fit)
# Check if the solution satisfies the constraints
print("Satisfaction :", rcpsp_problem.satisfy(best_solution))
# Evaluation :
print("Evaluation :", rcpsp_problem.evaluate(best_solution))

### Plot the solution

In [ ]:
from discrete_optimization.rcpsp.rcpsp_utils import plot_ressource_view

plot_ressource_view(rcpsp_model=rcpsp_problem, rcpsp_sol=best_solution)
# plot_task_gantt(rcpsp_model=rcpsp_problem, rcpsp_sol=best_solution)

Linear programming is a suitable solving approach for scheduling approaches, though it can lead to some bottlenecks when number of variables increase : here the time indexed approach would lose efficiency when the horizon is one order of magnitude higher !
Other linear formulation could be implemented that has better scaling property.